In [1]:
import sys
import os

# append the path of the current notebook to the sys.path
cur_dir = os.path.abspath(os.path.curdir)
root_dir = os.path.abspath(os.path.join(cur_dir, '..'))
sys.path.append(root_dir)

In [2]:
import pandas as pd

In [3]:
from src.data.load_data import load_data
from src.features.engineering import engineer_features
from src.models.evaluate import evaluate_model
from src.data.split_data import split_data
from sklearn.ensemble import RandomForestClassifier
from src.models.RandomForest import train_RandomForest
from src.features.selection import select_features

In [4]:
data = engineer_features(load_data(), 'readmitted', one_hot=True)
col = data.select_dtypes(include='uint8').columns

/home/hasi/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


--------------------
Feature engineering complete.
--------------------


In [5]:
data.astype('float64').dtypes

time_in_hospital      float64
num_lab_procedures    float64
num_procedures        float64
num_medications       float64
number_outpatient     float64
                       ...   
A1Cresult_None        float64
A1Cresult_Norm        float64
change_No             float64
diabetesMed_Yes       float64
readmitted            float64
Length: 503, dtype: object

In [6]:
data.shape

(101766, 47)

In [6]:
# decision tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [7]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

# Extract categorical columns (replace 'categorical_columns' with the actual names of your categorical columns)
# Replace with your categorical column names
X_categorical = X.select_dtypes(include=['object'])
cat_cols = X_categorical.columns

# Initialize the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform the categorical columns
X_encoded = ordinal_encoder.fit_transform(X_categorical)

# Replace the categorical columns in X with the encoded values
X.loc[:, cat_cols] = X_encoded

In [8]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

# one hot encoding
# Initialize the OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

# Fit and transform the categorical columns
X_encoded = one_hot_encoder.fit_transform(X)

In [9]:
data_enc = data.apply(LabelEncoder().fit_transform)

In [10]:
data_enc

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,repaglinide,...,diag_3_V8,max_glu_serum_>300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm,change_No,diabetesMed_Yes,readmitted
0,0,40,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
1,2,58,0,17,0,0,0,8,0,0,...,0,0,1,0,0,1,0,0,1,1
2,1,10,5,12,2,0,1,5,0,0,...,0,0,1,0,0,1,0,1,1,0
3,1,43,1,15,0,0,0,6,0,0,...,0,0,1,0,0,1,0,0,1,0
4,0,50,0,7,0,0,0,4,0,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,2,50,0,15,0,0,0,8,1,0,...,0,0,1,0,1,0,0,0,1,1
101762,4,32,3,17,0,0,1,8,0,0,...,0,0,1,0,0,1,0,1,1,0
101763,0,52,0,8,1,0,0,12,1,0,...,0,0,1,0,0,1,0,0,1,0
101764,9,44,2,20,0,0,1,8,0,0,...,0,0,1,0,0,1,0,0,1,0


In [11]:
dtree = DecisionTreeClassifier()

dtree.fit(data.drop('readmitted', axis=1), data['readmitted'])

DecisionTreeClassifier()

In [12]:
data.dtypes

time_in_hospital      float64
num_lab_procedures    float64
num_procedures        float64
num_medications       float64
number_outpatient     float64
                       ...   
A1Cresult_None        float64
A1Cresult_Norm        float64
change_No             float64
diabetesMed_Yes       float64
readmitted              int64
Length: 503, dtype: object

In [38]:
x = 1